In [1]:
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', -1)

In [5]:
%%time
col = ['year', 'month', 'url']
top_df = pd.DataFrame(columns=col)
#url = input("http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1")
url = "http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1"
path='https://www.e-stat.go.jp'
r = re.get(url)
data = r.text
year='None'
soup = BeautifulSoup(data, "lxml")
table = soup.find('div', {'class': 'stat-cycle_sheet'})
for year_section in table.find_all('ul', {'class': 'stat-cycle_ul_other'}):
    header = year_section.find('li', {'class': 'stat-cycle_header'})
    year = header.find('span').get_text(' ', strip=True)
    #   print('year changed to ' + year)
    months_section = year_section.find('li', {'class': 'stat-cycle_item'})
    for month_row in months_section.find_all('div'):
        month_a = month_row.find('a')
        month = month_a.get_text().rstrip('.\n')
        monthurl = path + month_a.get('href')
        row = pd.DataFrame([[year, month, monthurl]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #why the hell doesn't df.append work inplace?? Didn't it always use to?
print(top_df)

     year month  \
0    2018  Jan    
1    2018  Feb    
2    2018  Mar    
3    2017  Jan    
4    2017  Feb    
5    2017  Mar    
6    2017  Apr    
7    2017  May    
8    2017  Jun    
9    2017  Jul    
10   2017  Aug    
11   2017  Sep    
12   2017  Oct    
13   2017  Nov    
14   2017  Dec    
15   2016  Jan    
16   2016  Feb    
17   2016  Mar    
18   2016  Apr    
19   2016  May    
20   2016  Jun    
21   2016  Jul    
22   2016  Aug    
23   2016  Sep    
24   2016  Oct    
25   2016  Nov    
26   2016  Dec    
27   2015  Jan    
28   2015  Feb    
29   2015  Mar    
..    ...  ...    
117  2008  Jul    
118  2008  Aug    
119  2008  Sep    
120  2008  Oct    
121  2008  Nov    
122  2008  Dec    
123  2007  Jan    
124  2007  Feb    
125  2007  Mar    
126  2007  Apr    
127  2007  May    
128  2007  Jun    
129  2007  Jul    
130  2007  Aug    
131  2007  Sep    
132  2007  Oct    
133  2007  Nov    
134  2007  Dec    
135  2006  Jan    
136  2006  Feb    
137  2006  M

Retrieve available months by parsing website

In [4]:
def retrieve_month_excels(year, month, murl):
    mcols = ['year','month', 'excel_num', 'excel_description', 'excel_url']
    mdf = pd.DataFrame(columns=mcols)

#     year='2016'
#     month='Jan'
#     murl = df['url'][0]

    mr = re.get(murl)
    mdata = mr.text
    msoup = BeautifulSoup(mdata, "lxml")
    counter=0
    for elem in msoup.find_all('tr'):
        links = elem.find_all('a')
        if len(links) != 0:
            for link in links:
                if link.img.get('alt') == 'Excel File':
                    num = elem.td.string.strip()

                    excel_description = 'Not described'
                    tablerow=elem.find_all('td')
                    for cell in tablerow:
                        if cell.get('class')[0] == 'topborder':
                            excel_description = cell.string.strip()

                    excel_url = path + link.get('href').strip('.')

                    mrow = pd.DataFrame([[year, month, num, excel_description, excel_url]], columns=mcols)
                    mdf = mdf.append(mrow, ignore_index=True)
    return(mdf)
    # print(len(mdf))

Define function to enter link for each month, and store the excel notebook links and descriptions within to a dataframe.

In [5]:
%%time
cols = ['year','month', 'excel_num', 'excel_description', 'excel_url']
excelref_df = pd.DataFrame(columns = cols)
for index, row in top_df.iterrows():
    excelref_df = excelref_df.append(retrieve_month_excels(row['year'], row['month'], row['url']), ignore_index=True)
print(excelref_df)

     year month excel_num  \
0    2017  Jan   1          
1    2017  Jan   2          
2    2017  Jan   3-1        
3    2017  Jan   3-2        
4    2017  Jan   3-3        
5    2017  Feb   1          
6    2017  Feb   2          
7    2017  Feb   3-1        
8    2017  Feb   3-2        
9    2017  Feb   3-3        
10   2017  Mar   1          
11   2017  Mar   2          
12   2017  Mar   3-1        
13   2017  Mar   3-2        
14   2017  Mar   3-3        
15   2017  Apr   1          
16   2017  Apr   2          
17   2017  Apr   3-1        
18   2017  Apr   3-2        
19   2017  Apr   3-3        
20   2017  May   1          
21   2017  May   2          
22   2017  May   3-1        
23   2017  May   3-2        
24   2017  May   3-3        
25   2017  Jun   1          
26   2017  Jun   2          
27   2017  Jun   3-1        
28   2017  Jun   3-2        
29   2017  Jun   3-3        
..    ...  ...   ...        
521  2007  Oct   1          
522  2007  Oct   2          
523  2007  Nov

Utilize the function retrieve_month_excels to retrieve all available excel sheets for every month in the top-level dataframe

In [7]:
exceldesccount_df = excelref_df.groupby(['excel_description']).agg(['count']).sort_values([('year', 'count')], ascending=False).ix[:,:1]
exceldesccount_df.columns=['count']
# exceldesccount_df.insert(0,'excel_description', exceldesccount_df.index)
exceldesccount_df.reset_index(inplace=True)
print(exceldesccount_df)

                                                                                                                                                                                     excel_description  \
0   Number of In-migrants from Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures                                                                                             
1   Number of Out-migrants to Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures                                                                                              
2   Number of Net-migration by Age (5-Year Age Group) and Sex for Japan and Prefectures (All nationalities and Japanese)                                                                                 
3   Number of Net-migration by Age (5-Year Age Group) and Sex for Japan and Prefectures                                                                                                         

Count of sheets of each description in all available data

In [8]:
%%time
print(excelref_df.head())
mdir = os.path.join('extracts', 'monthly_report')
months = excelref_df.month.unique() #do not sort, retain the order
monthnum = list(range(1,13,1))
monthdirs = [str(num).zfill(2) + month for num,month in zip(monthnum, months)]
monthnamedict = dict(zip(months, monthdirs))
print(monthnamedict)

for index,row in excelref_df.iterrows():
    filedir = os.path.join(mdir, row['year'], monthnamedict.get(row['month']))
    filename = str(row['excel_num']) + '.xls'
    savepath = os.path.join(filedir, filename)
    if not os.path.isfile(savepath):
        os.makedirs(filedir, exist_ok=True)
        fileresponse = re.get(row['excel_url'], stream=True)
        with open(savepath, 'wb') as out_file:
            out_file.write(fileresponse.content)
        del fileresponse
        print("saved " + savepath)

   year month excel_num  \
0  2017  Jan   1          
1  2017  Jan   2          
2  2017  Jan   3-1        
3  2017  Jan   3-2        
4  2017  Jan   3-3        

                                                                                                                                                                                    excel_description  \
0  Number of Intra-prefectural Migrants, In-migrants from Other Prefectures and Out-migrants to Other Prefectures by Sex for Japan, Prefectures and 21 Major Cities (All nationalities and  Japanese)   
1  Number of Inter-prefectural Migrants by Sex and Origin for Prefectures, Tokyo Area, Nagoya Area, Osaka Area and 21 Major Cities (Japanese)                                                           
2  Number of In-migrants from Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures (Japanese)                                                                                  
3  Number of Out-migrants to Othe

Save all files available to disk if they don't already exist

In [14]:
samplecols = ['sampleyear', 'samplemonth', 'sampleexcel_num', 'excel_description', 'count']
samplebydesc_df = pd.DataFrame(columns=samplecols)
samplebydesc_df['excel_description'] = exceldesccount_df['excel_description']
samplebydesc_df['count'] = exceldesccount_df['count']

I want to build a dataframe of example excel workbooks of each description, so that i can formulate a way to parse each workbook according to their description.

In [15]:
# sampledesc = 'Number of In-migrants from Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures'
# dfdf1 = excelref_df[excelref_df['excel_description'] == sampledesc][:1]['month'].iloc[0]
samplebydesc_df['sampleyear'] = [excelref_df[excelref_df['excel_description'] == x][:1]['year'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
samplebydesc_df['samplemonth'] = [excelref_df[excelref_df['excel_description'] == x][:1]['month'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
samplebydesc_df['sampleexcel_num'] = [excelref_df[excelref_df['excel_description'] == x][:1]['excel_num'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
print(samplebydesc_df[:20])

   sampleyear samplemonth sampleexcel_num  \
0   2014       Jan         3-1              
1   2014       Jan         3-2              
2   2017       Jan         3-3              
3   2013       Jan         3-3              
4   2017       Jan         2                
5   2017       Jan         1                
6   2017       Jan         3-1              
7   2017       Jan         3-2              
8   2014       Jan         2                
9   2009       Jan         2                
10  2009       Jan         1                
11  2012       Jan         2                
12  2012       Jan         1                
13  2013       Jan         1                
14  2014       Jan         1                
15  2006       Jan         1                
16  2010       Jan         1                
17  2006       Jan         2                
18  2009       Apr         2                
19  2010       Jan         2                

                                                      

The first excel workbook with the matching description is assigned as the sample workbook